# 使用Python建置员工流失预警模型

## 商业理解

大数据使企业能够确定变量，预测自家公司的员工离职率。 
——《哈佛商业评论》2017年8月

员工流失分析就是评估公司员工流动率的过程，目的是预测未来的员工离职状况，减少员工流失情况。     
——《福布斯》2016年3月

企业培养人才需要大量的成本，为了防止人才再次流失，应当注重员工流失分析。员工流失分析是评估公司员工流动率的过程，目的是找到影响员工流失的主要因素，预测未来的员工离职状况，减少重要价值员工流失情况。

## 数据理解

数据取自于kaggle平台分享的数据集，共有10个字段14999条记录。 数据主要包括影响员工离职的各种因素（员工满意度、绩效考核、参与项目数、平均每月工作时长、工作年限、是否发生过工作差错、5年内是否升职、部门、薪资）以及员工是否已经离职的对应记录。 字段说明如下：

字段|字段翻译|角色|测量类型|不同值个数
---|:--:|---:|--:|--:
satisfaction_level|员工满意度|输入|连续|92
last_evaluation|最新绩效考核|输入|连续|65
number_project|参与项目数|输入|分类|6
average_montly_hours|平均每月工作时长|输入|连续|215
time_spend_company|工作年限|输入|分类|8
Work_accident|是否发生过工作差错|输入|分类|2
promotion_last_5years|5年内是否升职|输入|分类|2
sales|部门|输入|分类|10
salary|薪资|输入|分类|3
left|是否离职|目标|分类|2

## 读入数据

In [1]:
# 导入包
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

from pyecharts.charts import Bar, Pie, Page
from pyecharts import options as opts 
from pyecharts.globals import SymbolType, WarningType
WarningType.ShowWarning = False

In [2]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [3]:
# 读入数据
df = pd.read_csv('HR_comma_sep.csv')
df.head() 

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction_level       14999 non-null float64
last_evaluation          14999 non-null float64
number_project           14999 non-null int64
average_montly_hours     14999 non-null int64
time_spend_company       14999 non-null int64
Work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
sales                    14999 non-null object
salary                   14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [51]:
# 查看缺失值
print(df.isnull().any().sum()) 

0


可以发现，数据质量良好，没有缺失数据。

## 探索性分析

### 描述性统计

In [52]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
satisfaction_level,14999.0,0.612834,0.248631,0.09,0.44,0.64,0.82,1.0
last_evaluation,14999.0,0.716102,0.171169,0.36,0.56,0.72,0.87,1.0
number_project,14999.0,3.803054,1.232592,2.00,3.00,4.00,5.00,7.0
average_montly_hours,14999.0,201.050337,49.943099,96.00,156.00,200.00,245.00,310.0
time_spend_company,14999.0,3.498233,1.460136,2.00,3.00,3.00,4.00,10.0
Work_accident,14999.0,0.144610,0.351719,0.00,0.00,0.00,0.00,1.0
left,14999.0,0.238083,0.425924,0.00,0.00,0.00,0.00,1.0
promotion_last_5years,14999.0,0.021268,0.144281,0.00,0.00,0.00,0.00,1.0


从上述描述性分析结果可以看出：

员工满意度：范围0.09~1, 中位数0.640, 均值0.613, 总体来说员工对公司比较满意；

最新绩效考核：范围0.36~1, 中位数0.72, 均值0.716, 员工平均考核水平在中等偏上；

参与项目数：范围2~7, 中位数4, 均值3.8, 平均参加项目数约4个；

平均每月工作时长：范围96~310小时, 中位数200, 均值201。

工作年限：范围2~10年, 中位数3, 均值3.5。

### 离职人数占比

In [112]:
# 产生数据
cal_num = df.left.value_counts() 
data_pair =[list(z) for z in zip(cal_num.index.astype('str').to_list(), cal_num.values.tolist())] 

# 条形图
bar = Bar(init_opts=opts.InitOpts(width='1350px', height='750px')) 
bar.add_xaxis(cal_num.index.astype('str').tolist())
bar.add_yaxis('', cal_num.values.tolist(), color=Faker.rand_color()) 
bar.set_global_opts(title_opts=opts.TitleOpts(title='left变量频次图'), 
                    legend_opts=opts.LegendOpts(orient='vertical', pos_top='15%', pos_right='2%'),
                    visualmap_opts=opts.VisualMapOpts(is_show=False, min_=cal_num.min(), max_=cal_num.max(), range_color=['#BF4C51', '#8CB9D0'])) 
# bar.set_colors(['', '#BF4C51'])
bar.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

In [113]:
# 饼图
pie = Pie(init_opts=opts.InitOpts(width='1350px', height='750px'))
pie.add('', data_pair, radius=['35%', '65%'])
pie.set_global_opts(title_opts=opts.TitleOpts('left变量百分比对比图'), 
                    legend_opts=opts.LegendOpts(orient='vertical', pos_top='15%', pos_right='2%'))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{d}%"))
pie.set_colors(['#8CB9D0', '#BF4C51'])
pie.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

In [114]:
print(df.left.value_counts())
print('-'*20) 
print(df.left.value_counts(normalize=True)) 

0    11428
1     3571
Name: left, dtype: int64
--------------------
0    0.761917
1    0.238083
Name: left, dtype: float64


In [115]:
page0 = Page()
page0.add(bar, pie)
page0.render('./html/员工离职人数占比.html') 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\html\\员工离职人数占比.html'

总共有14999人，其中离职人数3571，占总人数的23.8%

### 员工满意度对离职的影响

In [56]:
df.groupby('left')['satisfaction_level'].describe() 

,count,mean,std,min,25%,50%,75%,max
left,,,,,,,,
0,11428.0,0.666810,0.217104,0.12,0.54,0.69,0.84,1.00
1,3571.0,0.440098,0.263933,0.09,0.13,0.41,0.73,0.92


In [57]:
def draw_numeric_graph(x_series, y_series, title):
    # 产生数据
    sat_cut = pd.cut(x_series, bins=25) 
    cross_table = round(pd.crosstab(sat_cut, y_series, normalize='index'),4)*100
    x_data = cross_table.index.astype('str').tolist()
    y_data1 = cross_table[cross_table.columns[1]].values.tolist()
    y_data2 = cross_table[cross_table.columns[0]].values.tolist() 
    
    # 条形图
    bar = Bar(init_opts=opts.InitOpts(width='1350px', height='750px'))
    bar.add_xaxis(x_data)
    bar.add_yaxis(str(cross_table.columns[1]), y_data1, stack='stack1', category_gap='0%')
    bar.add_yaxis(str(cross_table.columns[0]), y_data2, stack='stack1', category_gap='0%')
    bar.set_global_opts(title_opts=opts.TitleOpts(title), 
                        xaxis_opts=opts.AxisOpts(name=x_series.name, name_location='middle', name_gap=30),
                        yaxis_opts=opts.AxisOpts(name='百分比', name_location='middle', name_gap=30, min_=0, max_=100),
                        legend_opts=opts.LegendOpts(orient='vertical', pos_top='15%', pos_right='2%'))
    bar.set_series_opts(label_opts=opts.LabelOpts(is_show=False), 
                        itemstyle_opts=opts.ItemStyleOpts(border_color='black', border_width=0.3))
    bar.set_colors(['#BF4C51', '#8CB9D0']) 
    
    return bar 

In [58]:
bar1 = draw_numeric_graph(df['satisfaction_level'], df['left'], title='满意度评分与是否离职')
bar1.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

从直方图可以看出，离职员工的满意度评分明显偏低，平均值为0.44。满意度低于0.126分的离职率为97.2%。可见提升员工满意度可以有效防止人员流失。

### 最新绩效考核与是否离职

In [59]:
df.groupby('left')['last_evaluation'].describe() 

,count,mean,std,min,25%,50%,75%,max
left,,,,,,,,
0,11428.0,0.715473,0.162005,0.36,0.58,0.71,0.85,1.0
1,3571.0,0.718113,0.197673,0.45,0.52,0.79,0.90,1.0


In [60]:
bar2 = draw_numeric_graph(df['last_evaluation'], df['left'], title='最新绩效考核与是否离职')
bar2.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

平均来看，绩效考核成绩在离职/未离职员工之间差异不大。在离职员工中，绩效考核低、能力不够和绩效考核较高但工作压力大、满意度低、对薪资不满意可能成为离职的原因。

### 平均每月工作时长与是否离职

In [61]:
df.groupby('left')['average_montly_hours'].describe() 

,count,mean,std,min,25%,50%,75%,max
left,,,,,,,,
0,11428.0,199.060203,45.682731,96.0,162.0,198.0,238.0,287.0
1,3571.0,207.419210,61.202825,126.0,146.0,224.0,262.0,310.0


In [62]:
bar3 = draw_numeric_graph(df['average_montly_hours'], df['left'], title='平均每月工作时长与是否离职')
bar3.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

从直方图可以看出，月工作时长正常的员工离职率最低。而工时过低、过高的员工离职人数最多。证明恰当的工作任务分配是非常重要的。

### 参与项目数与是否离职

In [63]:
def draw_categorical_graph(x_series, y_series, title):
    # 产生数据
    cross_table = round(pd.crosstab(x_series, y_series, normalize='index'),4)*100
    x_data = cross_table.index.to_list()
    y_data1 = cross_table[1].values.tolist()
    y_data2 = cross_table[0].values.tolist() 
    
    # 条形图
    bar2 = Bar(init_opts=opts.InitOpts(width='1350px', height='750px'))
    bar2.add_xaxis(x_data) 
    bar2.add_yaxis(str(cross_table.columns[1]), y_data1, stack='stack1')
    bar2.add_yaxis(str(cross_table.columns[0]), y_data2, stack='stack1')
    bar2.set_global_opts(title_opts=opts.TitleOpts(title=title), 
                        yaxis_opts=opts.AxisOpts(name=x_series.name, name_location='middle', name_gap=40),
                        xaxis_opts=opts.AxisOpts(name='百分比', name_location='middle', name_gap=30, min_=0, max_=100),
                        legend_opts=opts.LegendOpts(orient='vertical', pos_top='15%', pos_right='2%')
                        )
    bar2.set_series_opts(label_opts=opts.LabelOpts(is_show=False, formatter='{c}%', position='inside'))
    bar2.set_colors(['#BF4C51', '#8CB9D0']) 
    bar2.reversal_axis()
    
    return bar2 

In [64]:
bar4 = draw_categorical_graph(df['number_project'], df['left'], title='参与项目数与是否离职')
bar4.render()  

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

从图中可以看出：

除项目数为2以外，随着项目数的增多，离职率在增大，且项目数是7的时候，离职率达到了100%以上。
综上两点，项目数2的离职率高，可能是这部分人工作能力不被认可。项目数6、7的总体少，离职率高，体现了他们的工作能力强，但同时工作压力太大导致他们离职。

### 员工工龄与是否离职

In [65]:
bar5 = draw_categorical_graph(df['time_spend_company'], df['left'], title='员工工龄与是否离职')
bar5.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

我们可以看到7年及以上工龄的员工基本没有离职，只有工龄为5年的员工离职人数超过在职人数。可见工龄长于6年的员工，由于种种原因，其“忠诚度”较高。而员工进入公司工作的第五5年是一个较为“危险”的年份，也许是该企业的“5年之痒”，应当重点关注该阶段的员工满意度、职业晋升等情况，以顺利过渡。

### 工作事故与是否离职

In [66]:
pd.crosstab(df['Work_accident'], df['left']) 

left,0,1
Work_accident,,
0,9428,3402
1,2000,169


In [67]:
bar6 = draw_categorical_graph(df['Work_accident'], df['left'], title='工作事故与是否离职')
bar6.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

从图中可看出，是否发生工作事故对员工离职的影响较小，可推测该企业处理工作事故的方式有可取之处。

### 员工晋升与是否离职

In [68]:
pd.crosstab(df['promotion_last_5years'], df['left']) 

left,0,1
promotion_last_5years,,
0,11128,3552
1,300,19


In [69]:
bar7 = draw_categorical_graph(df['promotion_last_5years'], df['left'], title='员工晋升与是否离职')
bar7.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

从条形图可以看出，在过去5年内获得未晋升的员工离职率为24.2%，比获得晋升的员工高4倍。设定良好的晋升通道可以很好的防止员工流失。

### 薪资水平与是否离职

In [70]:
pd.crosstab(df['salary'], df['left']) 

left,0,1
salary,,
high,1155,82
low,5144,2172
medium,5129,1317


In [71]:
bar8 = draw_categorical_graph(df['salary'], df['left'], title='薪资水平与是否离职')
bar8.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

可明显看出，薪资越高离职人数越少。证明为了减少离职率，提升员工福利待遇是一个可行的手段。

### 不同部门与是否离职

In [72]:
pd.crosstab(df['sales'], df['left']) 

left,0,1
sales,,
IT,954,273
RandD,666,121
accounting,563,204
hr,524,215
management,539,91
marketing,655,203
product_mng,704,198
sales,3126,1014
support,1674,555


In [73]:
bar9 = draw_categorical_graph(df['sales'], df['left'], title='不同部门与是否离职')
bar9.render() 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\render.html'

可见各部门离职率如上图，离职率由高到低分别为：人力部、财务部、科技部、支持部、销售部、市场部、IT部门、产品部、研发部、管理部。对于离职率过高的部门，应进一步分析关键原因。

In [74]:
page1 = Page() 
page1.add(bar1, bar2, bar3, bar4, bar5, bar6, bar7, bar8, bar9)  
page1.render('./html/员工流失因素分析.html') 

'C:\\Users\\wzd\\Desktop\\CDA\\CDA_Python\\Python项目实作\\数据建模\\员工流失\\html\\员工流失因素分析.html'

## 5. 数据预处理

由于sklearn在建模时不接受类别型变量，我们主要对数据做以下处理，以方便后续建模分析：

1. 薪资水平salary为定序变量, 因此将其字符型转化为数值型。
2. 岗位是定类型变量, 对其进行one-hot编码。

In [48]:
# 数据转换
df['salary'] = df['salary'].map({"low": 0, "medium": 1, "high": 2})
# 哑变量
df_dummies = pd.get_dummies(df, prefix='sales')
df_dummies.head() 

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0


## 6. 建模分析

我们使用决策树和随机森林进行模型建置，首先导入所需包：

In [75]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, roc_curve, plot_roc_curve

然后划分训练集和测试集，采用分层抽样方法划分80%数据为训练集，20%数据为测试集。

In [50]:
x = df_dummies.drop('left', axis=1)
y = df_dummies['left']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2020)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) 

(11999, 18) (3000, 18) (11999,) (3000,)


### 6.1 决策树

我们使用决策树进行建模，设置特征选择标准为gini，树的深度为5。输出分类的评估报告：

In [51]:
# 训练模型
clf = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=25)
clf.fit(X_train, y_train)
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)  

print('训练集：', classification_report(y_train, train_pred))
print('-' * 60) 
print('测试集：', classification_report(y_test, test_pred))

训练集：               precision    recall  f1-score   support

           0       0.98      0.99      0.98      9142
           1       0.97      0.93      0.95      2857

    accuracy                           0.98     11999
   macro avg       0.97      0.96      0.97     11999
weighted avg       0.98      0.98      0.97     11999

------------------------------------------------------------
测试集：               precision    recall  f1-score   support

           0       0.98      0.99      0.98      2286
           1       0.97      0.93      0.95       714

    accuracy                           0.98      3000
   macro avg       0.97      0.96      0.97      3000
weighted avg       0.98      0.98      0.98      3000



假设我们关注的是1类（即离职类）的F1-score,可以看到训练集的分数为0.95，测试集分数为0.95。

In [50]:
# 重要性
imp = pd.DataFrame([*zip(X_train.columns,clf.feature_importances_)], columns=['vars', 'importance'])
imp.sort_values('importance', ascending=False)  
imp = imp[imp.importance!=0]
imp 

,vars,importance
0,satisfaction_level,0.528842
1,last_evaluation,0.147619
2,number_project,0.099805
3,average_montly_hours,0.066193
4,time_spend_company,0.157450
17,sales_technical,0.000091


在属性的重要性排序中，员工满意度最高，其次是最新的绩效考核、参与项目数、每月工作时长。

然后使用网格搜索进行参数调优。

In [60]:
parameters = {'splitter':('best','random'),
              'criterion':("gini","entropy"),
              "max_depth":[*range(1, 20)],
             }

clf = DecisionTreeClassifier(random_state=25)
GS = GridSearchCV(clf, parameters, cv=10)
GS.fit(X_train, y_train)

print(GS.best_params_)

print(GS.best_score_) 

{'criterion': 'gini', 'max_depth': 15, 'splitter': 'best'}
0.9800813177648042


使用最优的模型重新评估训练集和测试集效果：

In [61]:
train_pred = GS.best_estimator_.predict(X_train)
test_pred = GS.best_estimator_.predict(X_test)

print('训练集：', classification_report(y_train, train_pred))
print('-' * 60) 
print('测试集：', classification_report(y_test, test_pred))

训练集：               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9142
           1       1.00      0.99      0.99      2857

    accuracy                           1.00     11999
   macro avg       1.00      0.99      1.00     11999
weighted avg       1.00      1.00      1.00     11999

------------------------------------------------------------
测试集：               precision    recall  f1-score   support

           0       0.99      0.98      0.99      2286
           1       0.95      0.97      0.96       714

    accuracy                           0.98      3000
   macro avg       0.97      0.98      0.97      3000
weighted avg       0.98      0.98      0.98      3000



可见在最优模型下模型效果有较大提升，1类的F1-score训练集的分数为0.99，测试集分数为0.96。

### 6.2 随机森林

下面使用集成算法随机森林进行模型建置，并调整max_depth参数。

In [67]:
rf_model = RandomForestClassifier(n_estimators=1000, oob_score=True, n_jobs=-1, 
                                  random_state=0)
parameters = {'max_depth': np.arange(3, 17, 1) }
GS = GridSearchCV(rf_model, param_grid=parameters, cv=10)
GS.fit(X_train, y_train)

print(GS.best_params_) 
print(GS.best_score_) 

{'max_depth': 16}
0.988582151793161


In [93]:
train_pred = GS.best_estimator_.predict(X_train)
test_pred = GS.best_estimator_.predict(X_test)

print('训练集：', classification_report(y_train, train_pred))
print('-' * 60) 
print('测试集：', classification_report(y_test, test_pred))

训练集：               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9142
           1       1.00      0.99      0.99      2857

    accuracy                           1.00     11999
   macro avg       1.00      1.00      1.00     11999
weighted avg       1.00      1.00      1.00     11999

------------------------------------------------------------
测试集：               precision    recall  f1-score   support

           0       0.99      1.00      0.99      2286
           1       0.99      0.97      0.98       714

    accuracy                           0.99      3000
   macro avg       0.99      0.99      0.99      3000
weighted avg       0.99      0.99      0.99      3000



可以看到在调优之后的随机森林模型中，1类的F1-score训练集的分数为0.99，测试集分数为0.98。

模型后续可优化方向：

1. 属性：数值型数据常常是模型不稳定的来源，可考虑对其进行分箱；重要属性筛选和字段扩充；
2. 算法：其他的集成方法；不同效能评估下的作法调整。